# Постановка задачи

### Задача

- Парсинг данных по книгам с сайта 'https://www.litres.ru/popular/' и построение модели пресказания потенциального рейтинга книги по ее текстовому описанию

### Описание данных

**Источник данных:** https://www.litres.ru/popular/

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
import numpy as np   
import pandas as pd          

# 4. Моделирование

## 4.1 Загрузка датасета

Загрузим обработанные данные

In [9]:
df = pd.read_pickle('../data/df_NLP.pkl')
display(df.head())
display(df.info())

,Наименование,Краткое описание,Рейтинг,Количество оценок,Дата публикации,Количество отзывов,page_link,new_raiting,Стемминг,Лемматизация
0,"Теория невероятности. Как мечтать, чтобы сбыва...","[никакой, магии, здравый, смысл, психология, в...",4.8,1541.0,2019-02-16,269.0,https://www.litres.ru/book/tatyana-muzhickaya/...,2.113456,никак маг здрав смысл психолог вер чудес книг ...,никакой магия здравый смысл психология вера чу...
1,"Читайте людей как книгу. Как анализировать, по...","[книге, патрика, кинга, автора, мировых, бестс...",4.2,456.0,2021-07-15,49.0,https://www.litres.ru/audiobook/patrik-king/ch...,1.709554,книг патрик кинг автор миров бестселлер област...,книга патрик кинг автор мировой бестселлер обл...
2,Перестаньте угождать людям. Будьте ассертивным...,"[угодничество, называют, болезнью, искалечить,...",4.3,160.0,2022-09-08,32.0,https://www.litres.ru/book/patrik-king/peresta...,1.646717,угодничеств называ болезн искалеч застав заб у...,угодничество называть болезнь искалечить заста...
3,Родная кровь,"[глубине, штата, берегу, залива, атлантическог...",4.8,8923.0,2022-01-12,639.0,https://www.litres.ru/book/anne-dar/rodnaya-kr...,3.392183,глубин штат берег залив атлантическ океа живет...,глубина штат берег залив атлантический океан ж...
4,Внутри убийцы,"[поймать, убийцу, нужно, смотреть, глазами, уб...",4.7,20230.0,2019-11-05,18000.0,https://www.litres.ru/book/mayk-omer/vnutri-ub...,3.410362,пойма убийц нужн смотрет глаз убийц бестселлер...,поймать убийца нужно смотреть глаз убийца бест...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1152 entries, 0 to 1151
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Наименование        1152 non-null   object        
 1   Краткое описание    1152 non-null   object        
 2   Рейтинг             1152 non-null   float64       
 3   Количество оценок   1152 non-null   float64       
 4   Дата публикации     1152 non-null   datetime64[ns]
 5   Количество отзывов  1152 non-null   float64       
 6   page_link           1152 non-null   object        
 7   new_raiting         1152 non-null   float64       
 8   Стемминг            1152 non-null   object        
 9   Лемматизация        1152 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(5)
memory usage: 90.1+ KB


None

## 4.2 train_test

Разделим наш датасет на обучающую и тестовую выборку

In [31]:
#train test_split
from sklearn.model_selection import train_test_split
train_df, test_df, train_y, test_y = train_test_split(df.drop(columns=['new_raiting']), df['new_raiting'], test_size=0.2, random_state=42)

##  4.3 Векторное представление Bag of Words

Используем метод **Bag of Words** для преобразования текста в признаковое пространство для использования класических моделей ML

### 4.3.1 Стемминг

Построим пространство признаков на базе результатов номализации типа Стемминг

In [32]:
#Инициализируем векторайзер
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 100)
vectorizer.fit(list(train_df['Стемминг']))

# Топ-10 слов
vectorizer.get_feature_names_out()[:10]

array(['автор', 'аудиокниг', 'бестселлер', 'бизнес', 'больш', 'важн',
       'ваш', 'вмест', 'возможн', 'войн'], dtype=object)

In [33]:
# Обучаем vectorizer на train-данных и сразу преобразем их в вектора с помощью метода fit_transform
stemming_train_X = vectorizer.transform(list(train_df['Стемминг']))
stemming_train_X.todense()[:2]

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [34]:
stemming_test_X = vectorizer.transform(list(test_df['Стемминг']))

### 4.3.2 Лемматизация

Построим пространство признаков на базе результатов номализации типа Лемматизация

In [35]:
#Инициализируем векторайзер
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 100)
vectorizer.fit(list(train_df['Лемматизация']))

# Топ-10 слов
vectorizer.get_feature_names_out()[:10]

array(['автор', 'аудиокнига', 'бестселлер', 'бизнес', 'быть', 'важный',
       'ваш', 'весь', 'война', 'вопрос'], dtype=object)

In [36]:
# Обучаем vectorizer на train-данных и сразу преобразем их в вектора с помощью метода fit_transform
lemm_train_X = vectorizer.transform(list(train_df['Лемматизация']))
lemm_train_X.todense()[:2]

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0,
         0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [37]:
lemm_test_X = vectorizer.transform(list(test_df['Лемматизация']))

### 4.3.3 Строим модель

In [17]:
#import алгоритма из библиотеки
from sklearn.ensemble import RandomForestRegressor

def get_fit_predict(X_train, y_train, X_test, ):
    # инициализируем модель
    model = RandomForestRegressor(n_estimators = 1000)

    # обучаем ее на тренировочных данных
    model.fit(X_train, list(y_train))

    # делаем предсказание для тестовых данных
    pred = model.predict(X_test)
    
    return pred

In [18]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

#### a) Стемминг

In [19]:
pred = get_fit_predict(X_train=stemming_train_X,
                       y_train=train_y,
                       X_test=stemming_train_X)
mean_absolute_percentage_error(train_y, pred)

6.98018838922464

In [20]:
pred = get_fit_predict(X_train=stemming_train_X,
                       y_train=train_y,
                       X_test=stemming_test_X)
mean_absolute_percentage_error(test_y, pred)

18.75846459096269

#### b)  Лемматизация

In [21]:
pred = get_fit_predict(X_train=lemm_train_X,
                       y_train=train_y,
                       X_test=lemm_train_X)
mean_absolute_percentage_error(train_y, pred)

7.046540215570077

In [22]:
pred = get_fit_predict(X_train=lemm_train_X,
                       y_train=train_y,
                       X_test=lemm_test_X)
mean_absolute_percentage_error(test_y, pred)

18.42057668347432

## 4.4 Векторное представление TF-IDF

Используем более сложный метод **TF-IDF** для преобразования текста в признаковое пространство для использования класических моделей ML

### 4.4.1 Стемминг

Построим пространство признаков на базе результатов номализации типа Стемминг

In [23]:
#вычисляем tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
# Fit TF-IDF on train texts
vectorizer = TfidfVectorizer(max_features = 200, norm = None) # возмем топ 200 слов
vectorizer.fit(list(train_df['Стемминг']))

# Топ-10 слов
vectorizer.get_feature_names_out()[:10]

array(['автор', 'аудиокниг', 'бестселлер', 'бизнес', 'больш', 'будущ',
       'важн', 'ваш', 'век', 'велик'], dtype=object)

In [24]:
# Обучаем TF-IDF на train, а затем применяем к train и test
stemming_train_X = vectorizer.fit_transform(list(train_df['Стемминг']))
stemming_test_X  = vectorizer.transform(list(test_df['Стемминг']))

### 4.4.2 Лемматизация

Построим пространство признаков на базе результатов номализации типа Лемматизация

In [25]:
#вычисляем tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
# Fit TF-IDF on train texts
vectorizer = TfidfVectorizer(max_features = 200, norm = None) # возмем топ 200 слов
vectorizer.fit(list(train_df['Лемматизация']))

# Топ-10 слов
vectorizer.get_feature_names_out()[:10]

array(['автор', 'аудиокнига', 'бестселлер', 'бизнес', 'будущее', 'быть',
       'важный', 'ваш', 'век', 'великий'], dtype=object)

In [26]:
# Обучаем TF-IDF на train, а затем применяем к train и test
lemm_train_X = vectorizer.fit_transform(list(train_df['Лемматизация']))
lemm_test_X  = vectorizer.transform(list(test_df['Лемматизация']))

### 4.4.3 Строим модель

#### a) Стемминг

In [27]:
pred = get_fit_predict(X_train=stemming_train_X,
                       y_train=train_y,
                       X_test=stemming_train_X)
mean_absolute_percentage_error(train_y, pred)

6.869461431941472

In [28]:
pred = get_fit_predict(X_train=stemming_train_X,
                       y_train=train_y,
                       X_test=stemming_test_X)
mean_absolute_percentage_error(test_y, pred)

18.454252801645165

#### b) Лемматизация

In [29]:
pred = get_fit_predict(X_train=lemm_train_X,
                       y_train=train_y,
                       X_test=lemm_train_X)
mean_absolute_percentage_error(train_y, pred)

6.770395882699337

In [30]:
pred = get_fit_predict(X_train=lemm_train_X,
                       y_train=train_y,
                       X_test=lemm_test_X)
mean_absolute_percentage_error(test_y, pred)

17.804974948024682

## 4.5 Поиск оптимальных парметров

In [80]:
import optuna
optuna.logging.set_verbosity(optuna.logging.ERROR)
from sklearn.model_selection import cross_val_score

In [84]:
def objective_RandomForest(trial):
    parameters_grid = {
        'criterion': trial.suggest_categorical('criterion', ['squared_error', 'absolute_error']),
        'n_estimators': trial.suggest_int('n_estimators', low=1000, high=1000, step=1),
        'max_samples': trial.suggest_categorical('max_samples', [None, 1.0, 0.5]),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', low=1, high=20, step=1),
        'min_samples_split': trial.suggest_int('min_samples_split', low=2, high=20, step=1),
    }

    model = RandomForestRegressor()
    X_local = lemm_train_X.copy()
    y_local = train_y.copy()

    # CV предсказание вероятности
    score = cross_val_score(model,
                              X_local,
                              y_local,
                              cv=3, scoring='neg_mean_absolute_percentage_error').mean()

    return score

In [85]:
study = optuna.create_study(direction="maximize")
study.optimize(objective_RandomForest, n_trials=30, n_jobs=-1, show_progress_bar=True)

print('Перечень подобранных параметров:')
study.best_params

  0%|          | 0/1 [00:00<?, ?it/s]

Перечень подобранных параметров:


{'criterion': 'absolute_error',
 'n_estimators': 1000,
 'max_samples': 1.0,
 'min_samples_leaf': 3,
 'min_samples_split': 13}

In [86]:
# инициализируем модель
model = RandomForestRegressor(**study.best_params)
# обучаем ее на тренировочных данных
model.fit(lemm_train_X, list(train_y))
# делаем предсказание для тестовых данных
pred = model.predict(lemm_test_X)

mean_absolute_percentage_error(test_y, pred)

18.270316514422127

# Заключение

В рамках работы:
 - Написан Парсер, с помощью которго была выгружена информация о книгах с сайта ЛитРес. Сформирован датасет.
 - Проведен обзорные EDA с приведеним данных к нужному формату;
 - Сформирован целевой признак из разных статистик, выгруженных с сайта.
 - Произведена токенизация и сформирован расширенный список стоп-слов из всех текстовых описаний.
 - Произведены два типа нормализации для дальнейшего сравнения.
 - Использованы два метода формирования признаково пространства и проведено моделирование обучающей и тестовой выборки для разных вариантов нормализации.
 
Результаты моделирования:
 - Наилучшие результаты получили при: TF-IDF с нормализацией типа Лемматизация